In [1]:
import numpy as np
import cv2
import os

# Function to generate custom dataset with geometric shapes
def generate_custom_shapes_dataset(num_samples_per_class=1000, output_dir='C:/Users/gsath/OneDrive/Desktop/Code/Jupyter Projects/dataset_created'):
    shapes = ['line', 'circle', 'ellipse', 'rectangle', 'rounded_rectangle', 'regular_polygon', 'star']
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for shape in shapes:
        shape_dir = os.path.join(output_dir, shape)
        if not os.path.exists(shape_dir):
            os.makedirs(shape_dir)

        for i in range(num_samples_per_class):
            img = np.zeros((128, 128), dtype=np.uint8)

            if shape == 'line':
                x1, y1 = np.random.randint(0, 128, size=2)
                x2, y2 = np.random.randint(0, 128, size=2)
                cv2.line(img, (x1, y1), (x2, y2), 255, 2)

            elif shape == 'circle':
                radius = np.random.randint(10, 40)
                x, y = np.random.randint(radius, 128-radius, size=2)
                cv2.circle(img, (x, y), radius, 255, 2)

            elif shape == 'ellipse':
                x, y = np.random.randint(20, 108, size=2)
                axes = np.random.randint(10, 50, size=2)
                angle = np.random.randint(0, 180)
                cv2.ellipse(img, (x, y), tuple(axes), angle, 0, 360, 255, 2)

            elif shape == 'rectangle':
                x1, y1 = np.random.randint(0, 108, size=2)
                x2, y2 = x1 + np.random.randint(20, 40), y1 + np.random.randint(20, 40)
                cv2.rectangle(img, (x1, y1), (x2, y2), 255, 2)

            elif shape == 'rounded_rectangle':
                x1, y1 = np.random.randint(0, 108, size=2)
                x2, y2 = x1 + np.random.randint(20, 40), y1 + np.random.randint(20, 40)
                radius = np.random.randint(5, 15)
                cv2.rectangle(img, (x1, y1), (x2, y2), 255, 2)
                cv2.circle(img, (x1, y1), radius, 255, 2)
                cv2.circle(img, (x2, y2), radius, 255, 2)
                cv2.circle(img, (x1, y2), radius, 255, 2)
                cv2.circle(img, (x2, y1), radius, 255, 2)

            elif shape == 'regular_polygon':
                points = np.random.randint(0, 128, size=(5, 2))
                cv2.polylines(img, [points], isClosed=True, color=255, thickness=2)

            elif shape == 'star':
                points = np.random.randint(0, 128, size=(5, 2))
                cv2.polylines(img, [points], isClosed=True, color=255, thickness=2)

            img_path = os.path.join(shape_dir, f'{i}.png')
            cv2.imwrite(img_path, img)

# Generate the custom dataset
generate_custom_shapes_dataset()

In [ ]:
import numpy as np
import cv2
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard


# Function to load images from a directory
def load_images_from_directory(base_dir):
    data = []
    labels = []
    shapes = ['line', 'circle', 'ellipse', 'rectangle', 'rounded_rectangle', 'regular_polygon', 'star']

    for shape in shapes:
        shape_dir = os.path.join(base_dir, shape)
        for filename in os.listdir(shape_dir):
            img_path = os.path.join(shape_dir, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                img = cv2.resize(img, (128, 128))
                data.append(img)
                labels.append(shape)

    return np.array(data), np.array(labels)

# Load dataset
data_dir = 'C:/Users/gsath/OneDrive/Desktop/Code/Jupyter Projects/dataset_created'
data, labels = load_images_from_directory(data_dir)

# Preprocess data
data = data.reshape((data.shape[0], 128, 128, 1)) / 255.0
label_encoder = LabelEncoder()
labels = to_categorical(label_encoder.fit_transform(labels))

# Split data
data_train, data_test, labels_train, labels_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Build the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(512, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# # Define callbacks
# early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

# Fit the model with callbacks
model.fit(data_train, labels_train, 
          epochs=20, 
          validation_split=0.2, 
          batch_size=32)
        #   callbacks=[early_stopping]

# Evaluate the model
loss, accuracy = model.evaluate(data_test, labels_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Save the model
model.save('./models/shapes_created_model.h5')

In [ ]:
import numpy as np
import cv2
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder

# Load the model
model = load_model("C:/Users/gsath/OneDrive/Desktop/Code/Jupyter Projects/models/shapes_created_model.h5")

# Load label encoder
label_encoder = LabelEncoder()
label_encoder.classes_ = np.array(['line', 'circle', 'ellipse', 'rectangle', 'rounded_rectangle', 'regular_polygon', 'star'])

# Function to preprocess images
def preprocess_image(image_path, target_size=(128, 128)):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is not None:
        img = cv2.resize(img, target_size)
        img = img.reshape((1, 128, 128, 1)) / 255.0
    return img

# Function to predict image class
def predict_image(model, image_path, label_encoder):
    img = preprocess_image(image_path)
    prediction = model.predict(img)
    predicted_class = label_encoder.inverse_transform([np.argmax(prediction)])
    return predicted_class[0]

# Function to display image with prediction
def display_image_with_prediction(image_path, predicted_class):
    img = cv2.imread(image_path)
    cv2.imshow(f'{predicted_class}', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Test with user-provided images
user_image_path = 'C:/Users/gsath/OneDrive/Desktop/Code/Jupyter Projects/problems/occlusion2_sol_rec.png'
predicted_class = predict_image(model, user_image_path, label_encoder)
print(f'Predicted Class: {predicted_class}')
display_image_with_prediction(user_image_path, predicted_class)